Entrenamiento de los 2 agentes

In [6]:
import numpy as np
import pickle
import random
import matplotlib.pyplot as plt

# Clase que define el entorno del juego, es decir, el tablero de Connect 4
class Ambiente:
    def __init__(self):
        # Define el tamaño del tablero (6 filas x 7 columnas)
        self.rows = 6
        self.columns = 7
        self.reset()  # Inicializa el tablero

    def reset(self):
        # Resetea el tablero y establece los valores iniciales
        self.board = np.zeros((self.rows, self.columns))
        self.turn = 1  # Jugador inicial (1: Rojo, -1: Amarillo)
        self.moves_made = 0  # Contador de movimientos
        self.done = False  # Indica si el juego ha terminado

    def is_valid_move(self, column):
        # Verifica si se puede hacer un movimiento en la columna (si la columna no está llena)
        return self.board[0][column] == 0

    def get_valid_moves(self):
        # Retorna una lista de columnas válidas (donde aún se puede jugar)
        return [col for col in range(self.columns) if self.is_valid_move(col)]

    def play_move(self, column):
        # Intenta jugar en la columna especificada, y retorna False si la columna está llena
        if not self.is_valid_move(column):
            return False
        # Coloca la ficha en la posición más baja disponible de la columna
        for row in range(self.rows - 1, -1, -1):
            if self.board[row][column] == 0:
                self.board[row][column] = self.turn  # Coloca la ficha
                self.moves_made += 1  # Incrementa el contador de movimientos
                break
        return True

    def switch_turn(self):
        # Cambia el turno del jugador (entre 1 y -1)
        self.turn *= -1

    def check_winner(self):
        # Revisa todas las posibles combinaciones de 4 en línea para determinar si hay un ganador
        for c in range(self.columns - 3):
            for r in range(self.rows):
                # Horizontal
                if np.abs(np.sum(self.board[r, c:c + 4])) == 4:
                    return True
        for c in range(self.columns):
            for r in range(self.rows - 3):
                # Vertical
                if np.abs(np.sum(self.board[r:r + 4, c])) == 4:
                    return True
        for c in range(self.columns - 3):
            for r in range(self.rows - 3):
                # Diagonal descendente
                if np.abs(np.sum([self.board[r + i, c + i] for i in range(4)])) == 4:
                    return True
                # Diagonal ascendente
                if np.abs(np.sum([self.board[r + 3 - i, c + i] for i in range(4)])) == 4:
                    return True
        return False

    def is_draw(self):
        # Verifica si el tablero está lleno, lo que indica un empate
        return self.moves_made == self.rows * self.columns

# Clase que define el agente que juega Connect 4 usando Q-learning
class Agente:
    def __init__(self, gamma=0.95, lr=0.01):
        self.q_table = {}  # Inicializa la tabla Q
        self.gamma = gamma  # Factor de descuento para valores futuros
        self.lr = lr  # Tasa de aprendizaje
        self.epsilon = 1.0  # Valor inicial de epsilon para exploración
        self.epsilon_decay = 0.999  # Decaimiento de epsilon
        self.epsilon_min = 0.05  # Valor mínimo de epsilon para mantener algo de exploración

    def get_state(self, env):
        # Convierte el tablero a una tupla para usarlo como estado en la Q-table
        return tuple(map(tuple, env.board))

    def choose_action(self, env):
        # Selecciona una acción usando una política epsilon-greedy
        state = self.get_state(env)
        if state not in self.q_table:
            # Inicializa los Q-valores para nuevos estados
            self.q_table[state] = [0] * env.columns
        if random.uniform(0, 1) < self.epsilon:
            # Explora una acción al azar
            return random.choice(env.get_valid_moves())
        else:
            # Explota el mejor movimiento basado en la Q-table
            return self.get_best_action(state, env)

    def get_best_action(self, state, env):
        # Retorna la mejor acción basada en los valores Q, penalizando las columnas inválidas
        valid_moves = env.get_valid_moves()
        q_values = np.array(self.q_table[state], dtype=float)
        q_values_invalid = [i for i in range(env.columns) if i not in valid_moves]
        q_values[q_values_invalid] = -1e6  # Asigna un valor bajo a las columnas llenas
        return np.argmax(q_values)

    def update_q_value(self, state, action, reward, next_state):
        # Actualiza el Q-valor de un estado y acción específicos
        if next_state not in self.q_table:
            self.q_table[next_state] = [0] * env.columns
        max_future_q = max(self.q_table[next_state])  # Máximo Q del siguiente estado
        current_q = self.q_table[state][action]
        # Fórmula de actualización de Q-Learning
        self.q_table[state][action] = current_q + self.lr * (reward + self.gamma * max_future_q - current_q)

    def save_q_table(self, file_name):
        # Guarda la Q-table en un archivo para cargarla más tarde
        with open(file_name, 'wb') as f:
            pickle.dump(self.q_table, f)

# Función para jugar una partida entre dos agentes
def jugar_partida(agente_rojo, agente_amarillo, env):
    env.reset()
    agentes = [agente_rojo, agente_amarillo]  # Lista de agentes, rojo y amarillo
    turno = 0  # Inicia con el agente rojo
    state_action_history = []  # Guarda el historial de estados y acciones

    while not env.done:
        agente_actual = agentes[turno]
        state = agente_actual.get_state(env)
        action = agente_actual.choose_action(env)
        state_action_history.append((state, action, turno))  # Guarda la acción tomada

        env.play_move(action)
        next_state = agente_actual.get_state(env)

        if env.check_winner():
            # Actualizar Q-valores para ambos agentes si hay un ganador
            for s, a, t in reversed(state_action_history):
                reward = 1 if t == turno else -1  # Recompensa para el ganador, castigo para el perdedor
                agentes[t].update_q_value(s, a, reward, next_state)
            env.done = True
            return 1 if turno == 0 else -1
        elif env.is_draw():
            # Actualizar Q-valores para ambos agentes en caso de empate
            for s, a, t in reversed(state_action_history):
                agentes[t].update_q_value(s, a, 0, next_state)  # Recompensa neutra en caso de empate
            env.done = True
            return 0
        else:
            # Actualizar Q-valores sin recompensa
            agentes[turno].update_q_value(state, action, 0, next_state)

        env.switch_turn()  # Cambiar el turno al otro agente
        turno = 1 - turno  # Alternar entre 0 y 1

# Código principal para entrenar a los agentes y guardar sus Q-tables
if __name__ == "__main__":
    env = Ambiente()  # Inicializa el entorno
    agente_rojo = Agente()  # Crea el agente rojo
    agente_amarillo = Agente()  # Crea el agente amarillo

    partidas = 100000  # Número de partidas para entrenar
    resultados = []  # Almacena los resultados de cada partida

    for i in range(partidas):
        resultado = jugar_partida(agente_rojo, agente_amarillo, env)
        resultados.append(resultado)
        # Reducir epsilon después de cada partida para más explotación y menos exploración
        agente_rojo.epsilon = max(agente_rojo.epsilon * agente_rojo.epsilon_decay, agente_rojo.epsilon_min)
        agente_amarillo.epsilon = max(agente_amarillo.epsilon * agente_amarillo.epsilon_decay, agente_amarillo.epsilon_min)
        if (i+1) % 1000 == 0:
            print(f"Partida {i+1}/{partidas} completada.")

    # Guardar las Q-tables de ambos agentes en archivos
    agente_rojo.save_q_table("q_table_rojo.pkl")
    agente_amarillo.save_q_table("q_table_amarillo.pkl")


Partida 1000/100000 completada.
Partida 2000/100000 completada.
Partida 3000/100000 completada.
Partida 4000/100000 completada.
Partida 5000/100000 completada.
Partida 6000/100000 completada.
Partida 7000/100000 completada.
Partida 8000/100000 completada.
Partida 9000/100000 completada.
Partida 10000/100000 completada.
Partida 11000/100000 completada.
Partida 12000/100000 completada.
Partida 13000/100000 completada.
Partida 14000/100000 completada.
Partida 15000/100000 completada.
Partida 16000/100000 completada.
Partida 17000/100000 completada.
Partida 18000/100000 completada.
Partida 19000/100000 completada.
Partida 20000/100000 completada.
Partida 21000/100000 completada.
Partida 22000/100000 completada.
Partida 23000/100000 completada.
Partida 24000/100000 completada.
Partida 25000/100000 completada.
Partida 26000/100000 completada.
Partida 27000/100000 completada.
Partida 28000/100000 completada.
Partida 29000/100000 completada.
Partida 30000/100000 completada.
Partida 31000/10000

In [4]:
# Contar y graficar resultados
victorias_rojo = resultados.count(1)
victorias_amarillo = resultados.count(-1)
empates = resultados.count(0)
# Crear gráfico de barras
labels = ['Rojo', 'Amarillo', 'Empate']
values = [victorias_rojo, victorias_amarillo, empates]
plt.bar(labels, values, color=['red', 'yellow', 'gray'])
plt.title('Distribución de Resultados')
plt.xlabel('Resultado')
plt.ylabel('Número de Partidas')
plt.show()

NameError: name 'resultados' is not defined

In [ ]:
import pickle

# Cargar el archivo .pkl
with open('q_table_rojo.pkl', 'rb') as file:
    data = pickle.load(file)

# Mostrar el contenido del archivo
print(data)

In [3]:
print("Número de estados en la Q-table:", len(agente_rojo.q_table))

NameError: name 'agente_rojo' is not defined

In [5]:
import tkinter as tk
import numpy as np
import pickle
import random
from tkinter import messagebox

# Clase que define el entorno del juego, es decir, el tablero de Connect 4
class Ambiente:
    def __init__(self):
        self.rows = 6  # Número de filas del tablero
        self.columns = 7  # Número de columnas del tablero
        self.reset()  # Inicializa el tablero

    def reset(self):
        # Crea una matriz de ceros para representar el tablero vacío
        self.board = np.zeros((self.rows, self.columns))
        self.turn = 1  # Turno del jugador actual (1 para rojo, -1 para amarillo)
        self.moves_made = 0  # Contador de movimientos realizados
        self.done = False  # Indica si el juego ha terminado

    def is_valid_move(self, column):
        # Verifica si se puede realizar un movimiento en la columna (si no está llena)
        return self.board[0][column] == 0

    def get_valid_moves(self):
        # Devuelve una lista de columnas donde es posible hacer un movimiento
        return [col for col in range(self.columns) if self.is_valid_move(col)]

    def play_move(self, column):
        # Realiza un movimiento en la columna especificada
        if not self.is_valid_move(column):
            return False  # Movimiento inválido si la columna está llena
        # Coloca la ficha en la posición más baja disponible en la columna
        for row in range(self.rows - 1, -1, -1):
            if self.board[row][column] == 0:
                self.board[row][column] = self.turn  # Coloca la ficha del jugador actual
                self.moves_made += 1  # Incrementa el contador de movimientos
                break
        return True

    def switch_turn(self):
        # Cambia el turno al otro jugador
        self.turn *= -1

    def check_winner(self):
        # Verifica si hay un ganador en el tablero

        # Comprobación horizontal
        for c in range(self.columns - 3):
            for r in range(self.rows):
                if np.abs(np.sum(self.board[r, c:c + 4])) == 4:
                    return True  # Hay un ganador

        # Comprobación vertical
        for c in range(self.columns):
            for r in range(self.rows - 3):
                if np.abs(np.sum(self.board[r:r + 4, c])) == 4:
                    return True  # Hay un ganador

        # Comprobación de diagonales
        for c in range(self.columns - 3):
            for r in range(self.rows - 3):
                # Diagonal descendente (\)
                if np.abs(np.sum([self.board[r + i, c + i] for i in range(4)])) == 4:
                    return True  # Hay un ganador
                # Diagonal ascendente (/)
                if np.abs(np.sum([self.board[r + 3 - i, c + i] for i in range(4)])) == 4:
                    return True  # Hay un ganador

        return False  # No hay ganador

    def is_draw(self):
        # Verifica si el juego es un empate (tablero lleno)
        return self.moves_made == self.rows * self.columns

# Clase que define el agente que juega usando una tabla Q
class Agente:
    def __init__(self, gamma=0.95, lr=0.01):
        self.q_table = {}  # Tabla Q para almacenar los valores Q
        self.gamma = gamma  # Factor de descuento
        self.lr = lr  # Tasa de aprendizaje
        self.epsilon = 0.05  # Pequeño valor para permitir algo de exploración

    def load_q_table(self, file_name):
        # Carga la tabla Q desde un archivo
        with open(file_name, 'rb') as f:
            self.q_table = pickle.load(f)

    def get_state(self, env):
        # Convierte el tablero en una representación inmutable (tupla) para usar como clave en la tabla Q
        return tuple(map(tuple, env.board))

    def choose_action(self, env):
        # Elige una acción usando una política ε-greedy
        state = self.get_state(env)
        if random.uniform(0, 1) < self.epsilon:
            # Explorar: elegir una acción válida al azar
            return random.choice(env.get_valid_moves())
        elif state in self.q_table:
            # Explotar: elegir la mejor acción conocida según la tabla Q
            return self.get_best_action(state, env)
        else:
            # Estado desconocido: seleccionar acción válida al azar
            return random.choice(env.get_valid_moves())

    def get_best_action(self, state, env):
        # Obtiene la mejor acción para un estado dado basado en la tabla Q
        valid_moves = env.get_valid_moves()
        q_values = np.array(self.q_table[state], dtype=float)
        # Penaliza las acciones inválidas asignándoles un valor muy bajo
        q_values[~np.isin(range(env.columns), valid_moves)] = -1e6
        return np.argmax(q_values)  # Retorna el índice de la acción con el valor Q más alto

# Clase que maneja la interfaz gráfica del juego
class Connect4GUI:
    def __init__(self, env, agente_rojo=None):
        self.env = env  # Entorno del juego
        self.agente_rojo = agente_rojo  # Agente que juega con fichas rojas
        self.window = tk.Tk()  # Ventana principal de Tkinter
        self.window.title("Conecta 4 - Humano (Amarillo) vs. Agente (Rojo)")

        # Botones de columna para que el jugador humano seleccione dónde colocar su ficha
        self.buttons = [
            tk.Button(
                self.window,
                text="▼",
                font=("Arial", 16, "bold"),
                bg="gold",
                command=lambda c=i: self.human_move(c)
            )
            for i in range(self.env.columns)
        ]
        # Colocar los botones en la ventana
        for i, button in enumerate(self.buttons):
            button.grid(row=0, column=i, sticky="nsew", padx=2, pady=2)

        # Canvas para dibujar el tablero del juego
        self.canvas = tk.Canvas(
            self.window,
            width=self.env.columns * 100,
            height=self.env.rows * 100,
            bg="#1E90FF"
        )
        self.canvas.grid(row=1, column=0, columnspan=self.env.columns, padx=5, pady=5)
        # Crear círculos que representan las posiciones en el tablero
        self.circles = [
            [
                self.canvas.create_oval(
                    j * 100 + 10,
                    i * 100 + 10,
                    j * 100 + 90,
                    i * 100 + 90,
                    fill="white"  # Color inicial de los círculos (vacíos)
                )
                for j in range(self.env.columns)
            ]
            for i in range(self.env.rows)
        ]

        # Configurar la distribución de columnas y filas en el grid
        for i in range(self.env.columns):
            self.window.grid_columnconfigure(i, weight=1)
        self.window.grid_rowconfigure(1, weight=1)
        self.turn = "yellow"  # Comienza el jugador humano (amarillo)

    def disable_buttons(self):
        # Deshabilita los botones para que el jugador no pueda realizar movimientos fuera de turno
        for button in self.buttons:
            button.config(state=tk.DISABLED)

    def enable_buttons(self):
        # Habilita los botones para que el jugador pueda realizar movimientos en su turno
        for button in self.buttons:
            button.config(state=tk.NORMAL)

    def human_move(self, column):
        # Maneja el movimiento del jugador humano
        if self.turn != "yellow":
            return  # Si no es el turno del jugador, no hacer nada
        if not self.env.is_valid_move(column):
            # Mostrar un mensaje si la columna está llena
            messagebox.showwarning("Columna llena", "Esta columna está llena. Elige otra.")
            return

        # Realizar el movimiento del jugador humano en el entorno
        self.env.play_move(column)
        # Encontrar la fila donde se colocó la ficha
        row = next(r for r in range(self.env.rows) if self.env.board[r][column] != 0)
        # Actualizar el círculo correspondiente en la interfaz gráfica
        self.canvas.itemconfig(self.circles[row][column], fill="yellow")

        if self.env.check_winner():
            # Si el jugador humano gana, mostrar mensaje y reiniciar el juego
            messagebox.showinfo("Fin del juego", "¡Ganaste!")
            self.reset_game()
            return
        elif self.env.is_draw():
            # Si hay empate, mostrar mensaje y reiniciar el juego
            messagebox.showinfo("Fin del juego", "¡Es un empate!")
            self.reset_game()
            return

        # Cambiar el turno al agente
        self.env.switch_turn()
        self.turn = "red"
        self.disable_buttons()  # Deshabilitar botones mientras juega el agente
        self.window.after(500, self.agent_play)  # Esperar medio segundo antes de que el agente juegue

    def agent_play(self):
        # Maneja el movimiento del agente
        if self.turn != "red" or not self.agente_rojo:
            return  # Si no es el turno del agente o no hay agente, no hacer nada

        # El agente elige una acción basada en su política
        action = self.agente_rojo.choose_action(self.env)
        # Realizar el movimiento del agente en el entorno
        self.env.play_move(action)
        # Encontrar la fila donde se colocó la ficha
        row = next(r for r in range(self.env.rows) if self.env.board[r][action] != 0)
        # Actualizar el círculo correspondiente en la interfaz gráfica
        self.canvas.itemconfig(self.circles[row][action], fill="red")

        if self.env.check_winner():
            # Si el agente gana, mostrar mensaje y reiniciar el juego
            messagebox.showinfo("Fin del juego", "¡El agente rojo ha ganado!")
            self.reset_game()
            return
        elif self.env.is_draw():
            # Si hay empate, mostrar mensaje y reiniciar el juego
            messagebox.showinfo("Fin del juego", "¡Es un empate!")
            self.reset_game()
            return

        # Cambiar el turno al jugador humano
        self.env.switch_turn()
        self.turn = "yellow"
        self.enable_buttons()  # Habilitar botones para que el jugador pueda hacer su movimiento

    def reset_game(self):
        # Reinicia el estado del juego y actualiza la interfaz gráfica
        self.env.reset()
        self.turn = "yellow"  # Comienza el turno del jugador humano
        for i in range(self.env.rows):
            for j in range(self.env.columns):
                # Restablecer el color de todos los círculos a blanco (vacío)
                self.canvas.itemconfig(self.circles[i][j], fill="white")
        self.enable_buttons()  # Habilitar botones para el nuevo juego

    def run(self):
        # Inicia el bucle principal de la interfaz gráfica
        self.window.mainloop()

if __name__ == "__main__":
    env = Ambiente()  # Crear el entorno del juego
    agente_rojo = Agente()  # Crear el agente rojo
    agente_rojo.load_q_table("q_table_rojo.pkl")  # Cargar la tabla Q entrenada del agente
    gui = Connect4GUI(env, agente_rojo)  # Crear la interfaz gráfica con el entorno y el agente
    gui.run()  # Ejecutar el juego
